In [169]:
import pandas as pd
import csv
df_dav = pd.read_csv('/content/df_davs.csv', sep=' ')
df_dav = df_dav.drop(["hate_speech", "count", "offensive_language", "neither"], axis = 1)
df_dav=df_dav.reindex(columns=['tweet', 'class'])
df_dav

,tweet,class
0,As a woman you shouldnt complain about cleani...,2
1,boy dats coldtyga dwn bad for cuffin dat hoe ...,1
2,Dawg RT You ever fuck a bitch and she start t...,1
3,she look like a tranny,1
4,The shit you hear about me might be true or i...,1
...,...,...
24778,yous a muthafin lie right His TL is trash Now ...,1
24779,youve gone and broke the wrong heart baby and ...,2
24780,young buck wanna eat dat nigguh like I aint fu...,1
24781,youu got wild bitches tellin you lies,1


In [170]:
import sklearn
from sklearn.model_selection import train_test_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [171]:
X_train, X_valid, Y_train, Y_valid= train_test_split(df_dav['tweet'].tolist(),
                                                      df_dav['class'].tolist(),
                                                      test_size=0.3)
train_data =list(zip(Y_train,X_train))
valid_data =list(zip(Y_valid,X_valid))

In [172]:
tzr = get_tokenizer('basic_english')
def torch_tokenizer(iterator):
  for _, text in iterator:
        yield tzr(text)

In [173]:
dataset_copy = train_data
transformed = build_vocab_from_iterator(torch_tokenizer(dataset_copy), specials=["<unk>"])
transformed.set_default_index(transformed["<unk>"])

In [174]:
text_to_index = lambda x: transformed(tzr(x))
label_trans = lambda x: int(x) 

In [175]:
text_to_index('hello world')

[1500, 414]

In [176]:
from torch import nn

class CNNModel1(nn.Module):

    def __init__(self, transformer_size, dim_in, dim_out):
        super(CNNModel1, self).__init__()
        self.embedding = nn.EmbeddingBag(transformer_size, dim_in, sparse=True)
        self.fc1 = nn.Linear(dim_in, 64)
        self.fc2 = nn.Linear(64,16)
        self.fc3 = nn.Linear(16, dim_out)
        self.relu = torch.nn.ReLU()
        self.init_weights()

    def init_weights(self):
        r = 0.5
        self.embedding.weight.data.uniform_(r, r)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)
        self.fc3.weight.data.uniform_(-r, r)
        nn.init.zeros_(self.fc3.bias)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = self.relu(self.fc1(embedded))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [177]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [178]:
train_iter1 = train_dat
dim_out = len(set([label for (label, text) in train_iter1]))
transformer_size = len(transformed)
dim_in = 128
model = CNNModel1(transformer_size, dim_in, dim_out).to(device)

In [179]:
 import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [180]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_trans(_label))
         processed_text = torch.tensor(text_to_index(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


In [190]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR =10  # learning rate
BATCH_SIZE = 16 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_dat
valid_iter2= valid_dat




train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1085 batches | accuracy    0.758
| epoch   1 |  1000/ 1085 batches | accuracy    0.768
-----------------------------------------------------------
| end of epoch   1 | time:  2.76s | valid accuracy    0.776 
-----------------------------------------------------------
| epoch   2 |   500/ 1085 batches | accuracy    0.762
| epoch   2 |  1000/ 1085 batches | accuracy    0.768
-----------------------------------------------------------
| end of epoch   2 | time:  2.66s | valid accuracy    0.776 
-----------------------------------------------------------
| epoch   3 |   500/ 1085 batches | accuracy    0.762
| epoch   3 |  1000/ 1085 batches | accuracy    0.768
-----------------------------------------------------------
| end of epoch   3 | time:  2.67s | valid accuracy    0.776 
-----------------------------------------------------------
| epoch   4 |   500/ 1085 batches | accuracy    0.762
| epoch   4 |  1000/ 1085 batches | accuracy    0.768
-------------------------

In [188]:
# from torch.utils.data.dataset import random_split
# from torchtext.data.functional import to_map_style_dataset
# EPOCHS = 10 # epoch
# LR = 10  # learning rate
# BATCH_SIZE = 16 # batch size for training

# def train_1(train_dat, valid_dat):
#   criterion = torch.nn.CrossEntropyLoss()
#   optimizer = torch.optim.SGD(model.parameters(), lr=LR)
#   scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
#   total_accu = None
#   train_iter2 = train_dat
#   valid_iter2= valid_dat
#   train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE, collate_fn=collate_batch)
#   valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

#   for epoch in range(EPOCHS):
#       train(train_dataloader)
#       accu_val = evaluate(valid_dataloader)
#       if total_accu is not None and total_accu > accu_val:
#         scheduler.step()
#       else:
#         total_accu = accu_val
#       print('Epoch {:3d}'
#             'valid accuracy {:8.3f} '.format(epoch, accu_val))

In [189]:
train_1(train_dat, valid_dat)

KeyboardInterrupt: ignored